In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import numpy as np
import pandas as pd
import time

import random

import psycopg2
import sqlalchemy
from sqlalchemy import create_engine

In [ ]:
DRIVER_PATH = r'Z:\РАБОЧИЕ ДОКУМЕНТЫ\1.БАРОМЕТР\СИСТЕМА  БАРОМЕТР Fine Report  2024\4. webdrivers\chromedriver_127.exe'

chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument("--headless")
# chrome_options.add_argument('start-maximized')
# chrome_options.add_argument('disable-infobars')
# chrome_options.add_argument('--disable-extensions')
# chrome_options.add_argument('--disable-client-side-phishing-detection')
# chrome_options.add_argument('--incognito')
# chrome_options.add_argument('--disable-background-networking')
# chrome_options.add_experimental_option( "prefs",{'profile.managed_default_content_settings.javascript': 2})

In [ ]:
driver = webdriver.Chrome(executable_path=DRIVER_PATH
                          ,options=chrome_options
                         )

In [ ]:
# collecting the list of listed companies on sse listed by their isic
# going to website
driver.get('https://www.sse.com.cn/assortment/stock/areatrade/trade/')

In [ ]:
# getting the info
SSE_ISIC_grouped = pd.DataFrame([elem.text for elem in driver.find_elements(By.CSS_SELECTOR, ".text-nowrap")])

# transforming the "1 by M" matrix/list into a "X by Y" matrix
SSE_ISIC_grouped = pd.DataFrame(np.reshape(SSE_ISIC_grouped, (int((len(SSE_ISIC_grouped))/6), 6)))

# getting and stitching on the column names
SSE_ISIC_grouped.columns = pd.DataFrame([elem.text for elem in driver.find_elements(By.CSS_SELECTOR, "th")]).T.iloc[0]

In [ ]:
# exporting for manual use
SSE_ISIC_grouped.to_excel(r'C:\Users\klyukin\Downloads\SSE_ISIC_grouped.xlsx', index=True)

In [ ]:
# gathering the individual company by each isic
ISIC_companies = pd.DataFrame()

for ticker in SSE_ISIC_grouped['行业代码']:
    # going to website
    driver.get(f'''https://www.sse.com.cn/assortment/stock/areatrade/trade/detail.shtml?csrcCode={ticker}''')

    # getting the info
    ISIC_companies_temp = pd.DataFrame([elem.text for elem in driver.find_elements(By.CSS_SELECTOR, ".text-nowrap")])
    
    # transforming the "1 by M" matrix/list into a "X by Y" matrix
    ISIC_companies_temp = pd.DataFrame(np.reshape(ISIC_companies_temp, (int((len(ISIC_companies_temp))/4), 4)))
    
    # getting and stitching on the column names
    ISIC_companies_temp.columns = pd.DataFrame([elem.text for elem in driver.find_elements(By.CSS_SELECTOR, "th")]).T.iloc[0]

    # adding the industry code from the previous step
    ISIC_companies_temp['行业代码'] = ticker

    # stitching the two together
    ISIC_companies = pd.concat([ISIC_companies, ISIC_companies_temp])

    # trying not to get flagged and banned by the stock exchange
    time.sleep(random.uniform(0, 1)+0.5)

In [ ]:
# # exporting for manual use
ISIC_companies.to_excel(r'C:\Users\klyukin\Downloads\ISIC_companies.xlsx', index=True)